In [ ]:
# In order to train model, first the training data needs to be made

# Therefore data before 2000 will be made into sentence and context, 
#but blame will only be assigned for the sentence in question

In [20]:
import spacy


In [ ]:
#load in data
import pandas as pd
df = pd.read_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")
#df['date'] = pd.to_datetime(df['date'])
#df.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/Corp_Folketing_V2.csv")
#df.head()

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country
0,1997-10-07,Dagsorden,1,Gert Petersen,NaN,NaN,True,191,Mødet er åbnet. I henhold til grundloven er Fo...,DK-Folketing,DNK
1,1997-10-07,Dagsorden,2,Formanden,NaN,NaN,True,182,"Jeg vil gerne takke Tinget for den tillid, man...",DK-Folketing,DNK
2,1997-10-07,Statsministerens redegørelse i henhold til gru...,3,Poul Nyrup Rasmussen,S,379.0,False,18662,For 25 år siden sagde et flertal i befolkninge...,DK-Folketing,DNK
3,1997-10-09,1) Indstilling fra Udvalget til Valgs Prøvelse.,2,Formanden,NaN,NaN,True,47,Fra Udvalget til Valgs Prøvelse har jeg modtag...,DK-Folketing,DNK
4,1997-10-09,2) Forhandling om redegørelse nr. R 1.,3,Torben Lund,S,379.0,False,2865,Vi står over for en meget afgørende folketings...,DK-Folketing,DNK


In [24]:
import re

def simplify_agenda(text):
    text = text.lower()  # make it case-insensitive
    text = text.strip()
    
    # Match anything containing "behandling af beslutningsforslag"
    if re.search(r'behandling af beslutningsforslag', text):
        return 'behandling af beslutningsforslag'
    
    elif 'forhandling' in text:
        return 'forhandling'
    
    else:
        return text
    
# Apply to your dataframe
df['agenda_simplified'] = df['agenda'].apply(simplify_agenda)


In [25]:
df['agenda_simplified'].value_counts()

agenda_simplified
forhandling                                                                                                                                                                                                                                                                       259655
behandling af beslutningsforslag                                                                                                                                                                                                                                                  184464
1) spørgsmål til ministrene.                                                                                                                                                                                                                                                       18711
til justitsministeren af:                                                                                                                  

In [18]:
from datetime import date
annotation_data = df[(df['date'].dt.date <= date(2000,1,1))]
annotation_data.to_csv("/work/MarkusLundsfrydJensen#1865/Bachelor_project/annotation_data.csv")

In [11]:
import random

# Example: annotation_data is your original DataFrame
paragraphs = annotation_data['text'].tolist()
original_indices = list(annotation_data.index)  # keep original index for para_id

# Shuffle paragraphs (zip together with original index)
zipped = list(zip(original_indices, paragraphs))
random.shuffle(zipped)

# Unzip
shuffled_indices, shuffled_paragraphs = zip(*zipped)


In [ ]:


# Load Danish pipeline
nlp = spacy.load("da_core_news_sm")

def split_paragraph(paragraph: str):
    doc = nlp(paragraph)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences


In [7]:
def build_samples(paragraph):
    sentences = split_paragraph(paragraph)
    samples = []
    
    for i, sent in enumerate(sentences):
        prev_sent = sentences[i-1] if i > 0 else ""
        next_sent = sentences[i+1] if i < len(sentences)-1 else ""
        
        #text = prev_sent + " [SEP] " + sent + " [SEP] " + next_sent

        sent_id = i

        # Three-part input with [SEP] handled by tokenizer later
        samples.append({
            "prev": prev_sent,
            "target": sent,
            "next": next_sent
        })
        
    return samples


In [8]:
def annotate_sentence(sentence, para_id, sent_id):
    """
    Annotate a sentence interactively.
    
    Returns:
        label (int) or None if escaped
    """
    print(f"\nPara {para_id}, Sent {sent_id}:")
    print(f"\"{sentence}\"")
    
    while True:
        user_input = input("Label this sentence (0 = no blame, 1 = blame, 'q' to quit): ").strip()
        if user_input.lower() == 'q':
            return None  # escape
        elif user_input in ['0', '1']:
            return int(user_input)
        else:
            print("Invalid input. Please enter 0, 1, or 'q'.")


In [ ]:
labelled_sentences = []

for para_id, para in zip(shuffled_indices, shuffled_paragraphs):
    samples = build_samples(para)  # build prev/target/next windows

    for sent_idx, s in enumerate(samples):
        # annotate interactively
        label = annotate_sentence(s['target'], para_id, sent_idx)
        if label is None:  # user pressed 'q'
            print("Annotation interrupted. Saving progress...")
            break
        
        labelled_sentences.append({
            "para_id": para_id,   # original index
            "sent_id": sent_idx,
            "prev": s['prev'],
            "target": s['target'],
            "next": s['next'],
            "label": label
        })
    else:
        continue  # only if inner loop wasn't broken
    break  # breaks outer loop if user quits



Para para_0, Sent 0:
"Mødet er åbnet."



Para para_0, Sent 1:
"I henhold til grundloven er Folketinget i dag  trådt sammen til sit første møde i det nye folketingsår."

Para para_0, Sent 2:
"Som det medlem, der længst har været medlem af Tinget,    påhviler det mig, jf. Folketingets forretningsordens § 2,  stk. 3, at lede valget af Tingets Præsidium og  tingsekretærer.       Til formand for Folketinget har samtlige Tingets  medlemmer indstillet hr. Erling Olsen, Socialdemokratiet."

Para para_0, Sent 3:
"Da  der således ikke foreligger andre indstillinger, vil jeg  betragte indstillingen som vedtaget."

Para para_0, Sent 4:
"(Ophold)."

Para para_0, Sent 5:
"Den er  vedtaget."

Para para_0, Sent 6:
"Til næstformænd har Tingets fire største partier - ud  over det parti, som formanden tilhører - udpeget følgende  medlemmer:       Til første næstformand hr. Ivar Hansen, Venstre, til  anden næstformand hr. Henning Grove, Det Konservative  Folkeparti, til tredje næstformand fru Margrete Auken,  Socialistisk Folkeparti, og til fje

In [10]:
#save data
data = pd.DataFrame(labelled_sentences)
data.to_csv("annotated_sentences.csv", index=False)

In [ ]:
#later when tokenizing:
#HuggingFace will automatically insert [SEP] between these segments.
#For BERT-like models, it also uses segment embeddings to distinguish the three parts.


def tokenize(batch):
    return tokenizer(
        batch["prev"], 
        batch["target"], 
        batch["next"], 
        truncation=True, 
        padding="max_length"
    )



In [ ]:
#here for the implementation (not ready yet I think just from chat)
#consider also saving the context
for para_id, paragraph in enumerate(paragraphs):
    sentences = sent_tokenize(paragraph, language="danish")
    for sent_id, sent in enumerate(sentences):
        label = blame_classifier(sent)  # 0 or 1
        if label == 1:
            results.append({"para_id": para_id, "sent_id": sent_id, "text": sent})
